# Dimostrazione automatica al primo ordine

In [1]:
#load "../../localInit.fsx"

open FolAutomReas.Formulas
open FolAutomReas.Fol
open FolAutomReas.Skolem
open FolAutomReas.Herbrand
open FolAutomReas.Lib
open FolAutomReas.Prop
open FolAutomReas.Pelletier

Nell'Handbook viene mostrato come la combinazione del teorema di Herbrand (ivi 3.23):

> Una formula priva di quantificatori $p$ è soddisfacibile al primo ordine 
> sse l’insieme di **tutte** le sue istanze ground è soddisfacibile (dal 
> punto di vista proposizionale).

più il teorema di compattezza (ivi 2.13) per la logica proposizionale:

> Per ogni insieme $\Gamma$ di formule proposizionali, se **ciascun** 
> sotto insieme **finito** $\Delta \subseteq \Gamma$ è soddisfacibile, 
> allora lo stesso Γ è soddisfacibile.

porta al seguente risultato ivi enunciato come Corollario 3.26:

> Una formula priva di quantificatori $p$ è insoddisfacibile al primo ordine
> sse **qualche** insieme **finito** di istanze ground è (proposizionalmente) 
> insoddisfacibile.

Questo risultato garantisce che una procedura in grado di generare per una formula $p$ insiemi sempre più grandi di istanze ground, e a condizione che ogni istanza ground della formula prima o poi compaia nell'enumerazione e che la formula stessa sia insoddisfacibile, prima o poi raggiungerà un insieme finito insoddisfacibile di formule proposizionali.

Questa impostazione da origine a una serie di procedure di semi-decisione circa l'insoddisfacibilità di una formula al primo ordine, perché, anche se tale formula contenesse dei quantificatori, sarebbe possibile fornirne una equi-soddisfacibile per mezzo della Skolemizzazione. D'altra parte, si parla di procedure di **semi-decisione** perché per una formula $p$ che invece fosse soddisfacibile questo processo potrebbe non terminare mai.

Si noti che avere procedure di semi-decisione circa l'insoddisfacibilità di una formula equivale ad averne anche per la la validità, perché una formula è valida se la sua negazione è insoddisfacibile.

## Skolemizzazione

Data una qualsiasi formula al primo ordine se ne può fornire una equi-soddisfacibile priva di quantificatori attraverso la funzione `skolemize` che 

* esegue una trasformazione in forma normale prenessa;
* sostituisce i quantificatori esistenziali con funzioni di skolem;
* elimina tutti i quantificatori universali perché la semantica presuppone già una quantificazione universale implicita.

Per esempio:

In [2]:
let fm = !!"~ (exists x. forall y. P(x) ==> P(y))"
let sfm = skolemize fm

display(fm)
display(sfm)

<<~(exists x. forall y. P(x) ==> P(y))>>

<<P(x) /\ ~P(f_y(x))>>

## Procedura di Gilmore

La procedura di Gilmore, partendo da una formula priva di quantificatori (espressa in dnf), genera insiemi sempre più grandi di istanze ground sotto forma di $m$-uple di termini ground dove $m$ è il numero di variabili (ovviamente libere) nella formula. Per garantire che qualsiasi istanza ground della formula prima o poi venga generata dalla procedura, un approccio utilizzato è quello di generare prima tutte le $m$-uple che coinvolgono solo simboli di funzione zero-ari (le costanti), poi quelli un-ari, poi i bin-ari, e così via...

Vediamo come questa procedura dimostri l'insoddisfacibilità della formula `fm` di sopra.

1. si parte dalla sua equi-soddisfacibile skolemizzata (e della dnf di questa si genereranno tutte le istanze ground):

In [8]:
let dnf = simpdnf sfm
dnf

[[<<P(x)>>;<<~P(f_y(x))>>]]

2. se ne estraggono le variabili, le costanti e le funzioni:

In [9]:
let fvs = fv sfm
let consts, funcs = herbfuns sfm

printfn "%s" (fvs.ToString())
printfn "%s" (consts.ToString())
printfn "%s" (funcs.ToString())

[x]
[(c, 0)]
[(f_y, 1)]


3. si iniziano a generare le $m$-uple di istanze ground di sole costanti:

In [10]:
let cntms = image (fun (c, _) -> Fn (c, [])) consts
let newtups = groundtuples cntms funcs 0 (List.length fvs)

let fl' = (distrib (image (image (subst (fpf fvs (newtups |> List.head)))) dnf) [[]])

fl'

[[<<P(c)>>;<<~P(f_y(c))>>]]

4. si testa se sono insoddisfacibili

In [11]:
fl'
|> List.filter (non trivial)
|> (<>) []
|> not

False

5. dal momento che non lo sono si generano le $m$-uple successive con funzioni un-arie:

In [12]:
let newtups' = groundtuples cntms funcs 1 (List.length fvs)
let fl'' = (distrib (image (image (subst (fpf fvs (newtups' |> List.head)))) dnf) [[]])
fl'' 

[[<<P(f_y(c))>>;<<~P(f_y(f_y(c)))>>]]

Dal momento che le due $m$-uple generate contengono due istanze complementari (`<<~P(f_y(c))>>` e `<<P(f_y(c))>>`), la procedura termina.